# "We rate dogs" data wrangling project

1. The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: [```twitter_archive_enhanced.csv```](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)

1. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (```image_predictions.tsv```) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

1. Each tweet's *retweet count* and *favorite ("like") count* at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's [Tweepy](http://www.tweepy.org/) library and store each tweet's entire set of JSON data in a file called ```tweet_json.txt``` file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

Note: do not include your Twitter API keys, secrets, and tokens in your project submission.


## Assessing Data for this Project

After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues. Detect and document at least **eight (8) quality issues** and **two (2) tidiness issues** in your ```wrangle_act.ipynb``` Jupyter Notebook. To meet specifications, the issues that satisfy the Project Motivation (see the Key Points header on the previous page) must be assessed.

## Cleaning Data for this Project

Clean each of the issues you documented while assessing. Perform this cleaning in ```wrangle_act.ipynb``` as well. The result should be a high quality and tidy master pandas DataFrame (or DataFrames, if appropriate). Again, the issues that satisfy the Project Motivation must be cleaned.

## Storing, Analyzing, and Visualizing Data for this Project

Store the clean DataFrame(s) in a CSV file with the main one named ```twitter_archive_master.csv```. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

Analyze and visualize your wrangled data in your ```wrangle_act.ipynb``` Jupyter Notebook. At least three (3) insights and one (1) visualization must be produced.

## Reporting for this Project

Create a 300-600 word written report called ```mwrangle_report.pdf``` or ```wrangle_report.html``` that briefly describes your wrangling efforts. This is to be framed as an internal document.

Create a 250-word-minimum written report called ```act_report.pdf``` or ```act_report.html``` that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

Both of these documents can be created in separate Jupyter Notebooks using the [Markdown functionality](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html) of Jupyter Notebooks, then downloading those notebooks as PDF files or HTML files (see image below). You might prefer to use a word processor like Google Docs or Microsoft Word, however.


In [51]:
import os
import tweepy
import requests
import pandas as pd
import numpy as np

## Gather Data

> #### From Template:  
> * Depending on the source of your data, and what format it's in, the steps in gathering data vary.  
> * High-level gathering process: obtaining data (downloading a file from the internet, scraping a web page,  
>   querying an API, etc.) and importing that data into your programming environment (e.g., Jupyter Notebook).  


Data for this projects originates from 3 sources:
1. A `twitter-archive-enhanced.csv` file, provided by Udacity
1. A `image-predictions.tsv` file, also provided by udacity, but downloaded with python `requests` function.
1. A `tweet_coverage.csv` file, created for this project with a twitter API.

In [52]:
def get_image_predictions():
    """Load image-predictions from disk if present, else load it from udacity.
    
    WARNING: This only works from within the udacity project workspace!
    
    """
    
    if  not os.path.exists('image-predictions.tsv'):
        r = requests.get("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv",
                         stream=True)
        tsv = r.raw.read()
        with open('image-predictions.tsv', 'wb') as f:
            f.write(tsv)
            
    df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
    
    return df_image_predictions

In [53]:
def get_tweet_coverate():
    """Load tweet_coverage from disk if present, else scrape it from twitter."""
    
    if  os.path.exists('tweet_coverage.csv'):
        df_tweet_coverage = pd.read_csv('tweet_coverage.csv')
    else:
        consumer_key = os.environ.get("TWITTER_API")
        consumer_secret = os.environ.get("TWITTER_API_SECRET")
        access_token = os.environ.get("TWITTER_ACCESS_TOKEN")
        access_secret = os.environ.get("TWITTER_ACCESS_TOKEN_SECRET")

        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)

        api = tweepy.API(auth)
        retweet_count = []
        favorite_count = []
        favorited = []
        retweeted = []

        for index, row in df_twitter_archive.iterrows():
            try:
                tweet_status = api.get_status(row.tweet_id, tweet_mode='extended')._json
                retweet_count.append(tweet_status['retweet_count'])
                favorite_count.append(tweet_status['favorite_count'])
                favorited.append(tweet_status['favorited'])
                retweeted.append(tweet_status['retweeted'])
            except:
                retweet_count.append(np.nan)
                favorite_count.append(np.nan)
                favorited.append(np.nan)
                retweeted.append(np.nan)

            if index % 100 == 0:
                print('Index is %d' % (index))
        print('ready')
        df_tweet_coverage = pd.DataFrame(data = list(zip(df_image_predictions['tweet_id'].values, retweet_count,
                                                         favorite_count, favorited, retweeted)),
                                         columns=['tweet_id', 'retweet_count', 
                                                  'favorite_count', 'favorited', 'retweeted'])
        df_tweet_coverage.to_csv('tweet_coverage.csv', index=False)        
        
    return df_tweet_coverage

In [54]:
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_image_predictions = get_image_predictions()
df_tweet_coverage = get_tweet_coverate()

## Asses Data

> #### From Template:
> * Assess data for:
>   * Quality: issues with content. Low quality data is also known as dirty data.
>   * Tidiness: issues with structure that prevent easy analysis. Untidy data is also known as messy data.
>     Tidy data requirements:
>       1. Each variable forms a column.
>       1. Each observation forms a row.
>       1. Each type of observational unit forms a table.
> 
> * Types of assessment:
>   * Visual assessment: scrolling through the data in your preferred software application (Google Sheets, Excel,
>     a text editor, etc.).
>   * Programmatic assessment: using code to view specific portions and summaries of the data (pandas' `head`, `tail`,
>     and `info` methods, for example).



In [100]:
# df_twitter_archive.head()
df_twitter_archive.sample(10)
#df_twitter_archive.expanded_urls[6]
#df_twitter_archive.info()
#df_twitter_archive[df_twitter_archive.expanded_urls.isna()]
#df_twitter_archive.doggo[0]
#df_twitter_archive.loc[0, 'doggo'] = None
#df_twitter_archive.name.astype('category')
#df_twitter_archive['name']
#df_twitter_archive.loc[22, 'text']
#df_twitter_archive.puppo.unique()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
589,799308762079035393,NaN,NaN,2016-11-17 17:50:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: I WAS SENT THE ACTUAL DOG IN TH...,7.743144e+17,4.196984e+09,2016-09-09 18:31:54 +0000,https://twitter.com/dog_rates/status/774314403...,14,10,None,None,None,None,None
1885,674793399141146624,6.717299e+17,4.196984e+09,2015-12-10 03:30:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have found another. 13/10 https://t.co/HwroP...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674793399...,13,10,None,None,None,None,None
328,833722901757046785,NaN,NaN,2017-02-20 17:00:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bronte. She's fairly h*ckin aerodynami...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/833722901...,13,10,Bronte,None,None,None,None
870,761672994376806400,NaN,NaN,2016-08-05 21:19:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ohboyohboyohboyohboyohboyohboyohboyohboyohboyo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/761672994...,10,10,None,None,None,None,None
1435,697259378236399616,NaN,NaN,2016-02-10 03:22:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending in saber-toothed tigers. T...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697259378...,8,10,getting,None,None,None,None
1635,684222868335505415,NaN,NaN,2016-01-05 04:00:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Someone help the girl is being mugged. Several...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684222868...,121,110,None,None,None,None,None
1289,708149363256774660,NaN,NaN,2016-03-11 04:35:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jebberson. He's the reigning hide and ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/708149363...,10,10,Jebberson,None,None,None,None
1105,734912297295085568,NaN,NaN,2016-05-24 01:02:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jax. He's a literal fluffball. Sneaky ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/734912297...,10,10,Jax,None,None,None,None
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None
2113,670434127938719744,NaN,NaN,2015-11-28 02:48:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Hank and Sully. Hank is very proud of the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670434127...,11,10,Hank,None,None,None,None


In [95]:
#df_image_predictions.head()
df_image_predictions.sample(10)
#df_image_predictions.info()
#df_image_predictions.p1.unique()


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1356,760539183865880579,https://pbs.twimg.com/media/Co36VZfWcAEN3R3.jpg,1,Samoyed,0.988013,True,malamute,0.004518,True,West_Highland_white_terrier,0.001189,True
811,692417313023332352,https://pbs.twimg.com/media/CZv13u5WYAA6wQe.jpg,1,bison,0.208922,False,mink,0.169945,False,polecat,0.144494,False
48,666826780179869698,https://pbs.twimg.com/media/CUELa0NUkAAscGC.jpg,1,Maltese_dog,0.359383,True,teddy,0.148759,False,West_Highland_white_terrier,0.106007,True
2054,888078434458587136,https://pbs.twimg.com/media/DFMWn56WsAAkA7B.jpg,1,French_bulldog,0.995026,True,pug,0.000932,True,bull_mastiff,0.000903,True
1371,762316489655476224,https://pbs.twimg.com/media/CpRKzZKWAAABGh7.jpg,1,African_grey,0.270468,False,Madagascar_cat,0.076187,False,television,0.033306,False
228,670385711116361728,https://pbs.twimg.com/media/CU2wPyWWUAAb1MJ.jpg,1,whippet,0.178027,True,Chesapeake_Bay_retriever,0.105969,True,beagle,0.078720,True
1775,828372645993398273,https://pbs.twimg.com/media/C374hb0WQAAIbQ-.jpg,1,malamute,0.663047,True,Eskimo_dog,0.207779,True,Tibetan_mastiff,0.040949,True
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,coho,0.404640,False,barracouta,0.271485,False,gar,0.189945,False
2040,885167619883638784,https://pbs.twimg.com/media/DEi_N9qXYAAgEEw.jpg,4,malamute,0.812482,True,Siberian_husky,0.071712,True,Eskimo_dog,0.055770,True
1003,708834316713893888,https://pbs.twimg.com/media/CdZI_bpWEAAm1fs.jpg,1,Eskimo_dog,0.283945,True,giant_panda,0.218252,False,malamute,0.180401,True


In [98]:
df_tweet_coverage.head()
df_tweet_coverage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 5 columns):
tweet_id          2075 non-null int64
retweet_count     879 non-null float64
favorite_count    879 non-null float64
favorited         879 non-null object
retweeted         879 non-null object
dtypes: float64(2), int64(1), object(2)
memory usage: 81.1+ KB


### Issues found in the files present:

#### Quality

1. In `df_twitter_archive` the **None** values in the columns `doggo`, `floofer`, `pupper` and `puppo` are strings.
1. In `df_twitter_archive` the name column always containts the word after "This is ...", which is not always the dogs name.
1. In `df_twitter_archive` the `retweeted_status_id` is float.
1. In `df_twitter_archive` the `retweeted_status_user_id` is float.
1. In `df_twitter_archive` the `timestamp`  column is of type object.
1. In `df_image_predictions` not all predicted dog breeds are actually dog breeds.
1. In `df_image_predictions` tweed id 762316489655476224 actaully is a dog, not a parrot (african grey).
1. In `df_tweet_coverage` retweet_count and facorite_count are float.
1. In `df_tweet_coverage` favorited and retweeted are objects.


#### Tidiness

1. In `df_twitter_archive` the columns doggo, floofer, pupper and puppo always contain "None" or the column name.
1. In `df_twitter_archive` the overall rating is not present, only nominator and denominator.
1. Data is spread arount three data frames (tables).

## Clean Data

> #### From Template:
> * Types of cleaning:
>   * Manual (not recommended unless the issues are single occurrences)
>   * Programmatic
> * The programmatic data cleaning process:
>   1. Define: convert our assessments into defined cleaning tasks. These definitions also serve as an instruction
>      list so others (or yourself in the future) can look at your work and reproduce it.
>   1. Code: convert those definitions to code and run that code.
>   1. Test: test your dataset, visually or with code, to make sure your cleaning operations worked.
> * Always make copies of the original pieces of data before cleaning!

### Define
#### Quality

1. In `df_twitter_archive` the **None** values in the columns `doggo`, `floofer`, `pupper` and `puppo` are strings.
   
   **Solution:** Change the column type to Boolean, this will also enhance the tidiness. Since according to [Dogtionary](https://video.udacity-data.com/topher/2017/October/59e04ceb_dogtionary-combined/dogtionary-combined.png) dogs can be "doggo" and "pupper" at the same time, the columns will not be merged to one categoracal column.
1. In `df_twitter_archive` the name column always containts the word after "This is ...", which is not always the dogs name.

   **Solution:** This can only be dealt with manually. Since the dog's name column containts faulty data I will erase it.
1. In `df_twitter_archive` the `retweeted_status_id` is float.

   **Solution:** Convert `retweeted_status_id`  column to int64.
1. In `df_twitter_archive` the `retweeted_status_user_id` is float.

   **Solution:** Convert `retweeted_status_user_id`  column to int64.
1. In `df_twitter_archive` the `timestamp` column is of type object.

   **Solution:** Convert `timestamp`  column to datetime.
1. In `df_image_predictions` not all predicted dog breeds are actually dog breeds.

   **Solution:** Erase all rows, that do not contain pictures of dogs, with a confidence `p1_conf` of less than 95%.
1. In `df_image_predictions` tweed id 762316489655476224 actaully is a dog, not a parrot (african grey).

   **Solution:** Erase all rows, that do not contain dogs (```p1_dog == FALSE```).
1. In `df_tweet_coverage` retweet_count and facorite_count are float.

   **Solution:** convert retweet_count and favorite_count to `int64`.
1. In `df_tweet_coverage` favorited and retweeted are objects.

   **Solution:** cast favorited and retweeted to `Bool`. 


#### Tidiness

1. In `df_twitter_archive` the columns doggo, floofer, pupper and puppo always contain "None" or the column name.

   **Solution:** Already taken care of in Quality section (cast to `Boolean`).
1. In `df_twitter_archive` the overall rating is not present, only nominator and denominator.

   **Solution:** Calcuate an overall rating and 

1. Data is spread arount three data frames (tables).

   **Solution:** Outer join all data frames with common tweet_id, reassess afterwards.




## Reassess (Test) and Iterate

* After cleaning, always reassess and iterate on any of the data wrangling steps if necessary.

## Store (Optional)

* Store data, in a file or database for example, if you need to use it in the future.

**Also store the dtypes.**